In [9]:
# Install required packages
!pip install requests pandas pytz

import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz

def fetch_declining_memecoins(api_key):
    """
    Fetches memecoins that:
    1. Created after October 2022
    2. At least one year old
    3. Show declining market cap
    """
    headers = {
        'X-CMC_PRO_API_KEY': api_key
    }

    try:
        # Calculate date ranges
        current_date = datetime.now(pytz.UTC)
        cutoff_start = pd.Timestamp('2022-10-01', tz=pytz.UTC)
        cutoff_end = current_date - timedelta(days=365)  # Must be at least 1 year old

        print("Fetching data from CoinMarketCap...")
        url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
        params = {
            'start': '1',
            'limit': '5000',
            'convert': 'USD',
            'sort': 'date_added',
            'sort_dir': 'desc',
            'cryptocurrency_type': 'tokens'
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        print("Analyzing tokens...")
        declining_memecoins = []

        for coin in data['data']:
            # Convert date_added to timezone-aware timestamp
            date_added = pd.to_datetime(coin['date_added'])
            if date_added.tzinfo is None:  # If naive, localize it
                date_added = date_added.tz_localize(pytz.UTC)
            else:  # If already aware, convert to UTC
                date_added = date_added.tz_convert(pytz.UTC)

            # Check if coin meets date criteria
            if not (cutoff_start <= date_added <= cutoff_end):
                continue

            # Check if it's a memecoin by tags
            tags = [tag.lower() for tag in coin.get('tags', [])]
            if not any(tag in tags for tag in ['meme', 'memes', 'dog', 'shiba', 'inu', 'pepe', 'elon']):
                continue

            # Check for declining trend
            percent_change_30d = coin['quote']['USD']['percent_change_30d']
            percent_change_90d = coin['quote']['USD']['percent_change_90d']

            # Only include if showing consistent decline
            if percent_change_30d is None or percent_change_90d is None:
                continue

            if percent_change_30d < 0 and percent_change_90d < 0:
                declining_memecoins.append({
                    'name': coin['name'],
                    'symbol': coin['symbol'],
                    'date_added': date_added.strftime('%Y-%m-%d'),
                    'market_cap': coin['quote']['USD']['market_cap'],
                    'price_usd': coin['quote']['USD']['price'],
                    'volume_24h': coin['quote']['USD']['volume_24h'],
                    'percent_change_30d': percent_change_30d,
                    'percent_change_90d': percent_change_90d,
                    'platform': coin.get('platform', {}).get('name', 'Unknown'),
                    'contract': coin.get('platform', {}).get('token_address', 'Unknown'),
                    'tags': ', '.join(coin.get('tags', [])),
                    'total_supply': coin.get('total_supply', 'Unknown'),
                    'circulating_supply': coin.get('circulating_supply', 'Unknown')
                })

        if not declining_memecoins:
            print("No matching memecoins found")
            return None

        # Create DataFrame
        df = pd.DataFrame(declining_memecoins)

        # Sort by largest market cap decline
        df = df.sort_values('percent_change_90d')

        # Save results
        filename = f'declining_memecoins_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        df.to_csv(filename, index=False)

        print(f"\nFound {len(df)} declining memecoins")
        print("\nTop 10 most declining memecoins:")
        display(df[['name', 'symbol', 'market_cap', 'percent_change_90d', 'date_added']].head(10))

        return df

    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        print(f"Full error: {traceback.format_exc()}")

        return None

# Mount Google Drive (optional - uncomment if you want to save results to Drive)
# from google.colab import drive
# drive.mount('/content/drive')

# Your API key
API_KEY = 'ENTER API KEY HERE'

print("Starting analysis...")
df = fetch_declining_memecoins(API_KEY)

if df is not None:
    print("\nDetailed Analysis:")
    print(f"Total tokens found: {len(df)}")
    print(f"\nAverage Market Cap: ${df['market_cap'].mean():,.2f}")
    print(f"Average 30-day decline: {df['percent_change_30d'].mean():.2f}%")
    print(f"Average 90-day decline: {df['percent_change_90d'].mean():.2f}%")
    print(f"\nMost common platform: {df['platform'].mode().iloc[0]}")
    print(f"Date range: {df['date_added'].min()} to {df['date_added'].max()}")


Starting analysis...
Fetching data from CoinMarketCap...
Analyzing tokens...

Found 76 declining memecoins

Top 10 most declining memecoins:


,name,symbol,market_cap,percent_change_90d,date_added
63,Champz,CHAMPZ,0.000000e+00,-99.999986,2023-06-14
50,Kento,KNTO,0.000000e+00,-91.562355,2023-08-07
32,Oggy Inu (ETH),OGGY,2.658725e+04,-90.260997,2023-10-26
38,Froggy,FROGGY,0.000000e+00,-87.240483,2023-09-28
9,Nola,NOLA,0.000000e+00,-76.930273,2023-12-19
12,Grok Girl,GROKGIRL,0.000000e+00,-76.862117,2023-12-16
17,CRAZY DOGE,CRAZYDOGE,0.000000e+00,-71.543941,2023-12-05
44,BABYTRUMP,BABYTRUMP,3.101707e+05,-71.504288,2023-08-31
39,Wall Street Memes,WSM,1.939358e+06,-69.236722,2023-09-26
73,MASYA,MASYA,0.000000e+00,-66.828288,2023-05-21



Detailed Analysis:
Total tokens found: 76

Average Market Cap: $18,985,456.60
Average 30-day decline: -31.78%
Average 90-day decline: -36.38%

Most common platform: Ethereum
Date range: 2023-05-20 to 2023-12-23
